In [74]:
import torch.nn as nn
import torch.optim as optim
import torch.nn as nn
from sentence_transformers import models
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertModel
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import DataLoader

In [98]:
# Using an already pretrained SBERT to compute cosine similarities which would then be used 

df=pd.read_csv("stsb_train.csv")
df=df.drop("idx",axis=1)
df_cosine_labels = df.copy() 
df_cosine_labels["label"] = df_cosine_labels["label"] /df_cosine_labels["label"].abs().max() 


df=pd.read_csv("stsb_validation.csv")
df=df.drop("idx",axis=1)
validation_cosine_labels = df.copy() 
validation_cosine_labels["label"] = validation_cosine_labels["label"] /validation_cosine_labels["label"].abs().max() 



In [99]:
train_examples = []
for i,row in df_cosine_labels.iterrows():
    train_examples.append([[row["sentence1"], row["sentence2"]],row["label"]])

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

val_examples = []
for i,row in validation_cosine_labels.iterrows():
    val_examples.append([[row["sentence1"], row["sentence2"]],row["label"]])

val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=16)

In [46]:

class Shroomformer(nn.Module):
    def __init__(self, sequence_length):
        super(Shroomformer, self).__init__()
        self.sequence_length = sequence_length
        self.word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=self.sequence_length)
        self.pooling_model = models.Pooling(self.word_embedding_model.get_word_embedding_dimension())
        self.twin1 = nn.Sequential(
            self.word_embedding_model,
            self.pooling_model
        )
        self.twin2 = nn.Sequential(
            self.word_embedding_model,
            self.pooling_model
        )
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
           
    def forward(self, sentence1_encoded, sentence2_encoded):
        output1 = self.twin1(sentence1_encoded)["sentence_embedding"]
        output2 = self.twin2(sentence2_encoded)["sentence_embedding"]
        similarity_score = self.cos(output1, output2)
        return similarity_score


def encode_pair(pair,padding=True,truncation=True,max_length=128):
    tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
    return tokenizer(
    pair[0], padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    ), tokenizer(
        pair[1],padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    )


model = Shroomformer(128)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs=500

from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences, labels = data
        optimizer.zero_grad()
        sentence1,sentence2=encode_pair(sentences)
        outputs = model(sentence1,sentence2)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print()
    print(f'Epoch {epoch+1} | Loss: {running_loss / len(train_dataloader)}')




  0%|          | 0/500 [00:00<?, ?it/s]

Batch: 1
Batch: 101
Batch: 201
Batch: 301


  0%|          | 1/500 [06:04<50:28:23, 364.13s/it]


Epoch 1 | Loss: 0.2966899295647939
Batch: 1
Batch: 101
Batch: 201
Batch: 301


  0%|          | 2/500 [11:48<48:46:37, 352.61s/it]

...


Epoch 100 | Loss: 0.29754847710331284
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 20%|██        | 101/500 [9:14:38<35:33:02, 320.76s/it]


Epoch 101 | Loss: 0.29719463781350186
Batch: 1


 20%|██        | 101/500 [9:15:00<36:32:33, 329.71s/it]


KeyboardInterrupt: 

In [47]:
torch.save(model.state_dict(), 'model_v1.pth')

In [84]:

class ShroomformerV2(nn.Module):
    def __init__(self):
        super(ShroomformerV2, self).__init__()
        self.twin1 = BertModel.from_pretrained("bert-base-uncased")
        self.twin2 = BertModel.from_pretrained("bert-base-uncased")
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    def mean_pooling(self,word_embeddings, attention_mask):
        token_embeddings = word_embeddings[0] 
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

  
    def forward(self, sentence1_encoded, sentence2_encoded):
        word_embeddings_1 = self.twin1(**sentence1_encoded)
        sentence_embeddings_1=self.mean_pooling(word_embeddings_1, sentence1_encoded["attention_mask"])
        word_embeddings_2 = self.twin2(**sentence2_encoded)
        sentence_embeddings_2=self.mean_pooling(word_embeddings_2, sentence2_encoded["attention_mask"])
        similarity_score = self.cos(sentence_embeddings_1,sentence_embeddings_2)
        return similarity_score
       


def encode_pair(pair,padding=True,truncation=True,max_length=128):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    return tokenizer(
    pair[0], padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    ), tokenizer(
        pair[1],padding=padding,truncation=truncation,max_length=max_length, return_tensors="pt"
    )


model = ShroomformerV2()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
num_epochs=100
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters in the model:", total_params)
model.train()
from tqdm import tqdm
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences, labels = data
        optimizer.zero_grad()
        sentence1,sentence2=encode_pair(sentences)
        outputs=model(sentence1,sentence2)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print()
    print(f'Epoch {epoch+1} | Loss: {running_loss / len(train_dataloader)}')




Total parameters in the model: 218964480


  0%|          | 0/100 [00:00<?, ?it/s]

Batch: 1
Batch: 101
Batch: 201
Batch: 301


  1%|          | 1/100 [06:58<11:30:45, 418.64s/it]


Epoch 1 | Loss: 0.09140385061295496
Batch: 1
Batch: 101
Batch: 201
Batch: 301


...


Epoch 34 | Loss: 0.08593545713358455
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 35%|███▌      | 35/100 [4:25:02<8:19:17, 460.88s/it]


Epoch 35 | Loss: 0.08628203681566649
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 36%|███▌      | 36/100 [4:32:09<8:00:59, 450.93s/it]


Epoch 36 | Loss: 0.085855706812193
Batch: 1
Batch: 101
Batch: 201


In [15]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    mps_device = torch.device("cpu")
    

else:
    mps_device = torch.device("mps")

In [91]:

class ShroomformerV2(nn.Module):
    def __init__(self):
        super(ShroomformerV2, self).__init__()
        self.twin1 = BertModel.from_pretrained("bert-base-uncased")
        self.twin2 = BertModel.from_pretrained("bert-base-uncased")
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    def mean_pooling(self,word_embeddings, attention_mask):
        token_embeddings = word_embeddings[0] 
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

  
    def forward(self, sentence1_encoded, sentence2_encoded):
        word_embeddings_1 = self.twin1(**sentence1_encoded)
        sentence_embeddings_1=self.mean_pooling(word_embeddings_1, sentence1_encoded["attention_mask"])
        word_embeddings_2 = self.twin2(**sentence2_encoded)
        sentence_embeddings_2=self.mean_pooling(word_embeddings_2, sentence2_encoded["attention_mask"])
        similarity_score = self.cos(sentence_embeddings_1,sentence_embeddings_2)
        return similarity_score
       


def encode_pair(pair, padding=True, truncation=True, max_length=128):
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    encoded_pair_1 = tokenizer(
        pair[0], 
        padding=padding, 
        truncation=truncation, 
        max_length=max_length, 
        return_tensors="pt"
    )
    encoded_pair_2 = tokenizer(
        pair[1],
        padding=padding,
        truncation=truncation,
        max_length=max_length,
        return_tensors="pt"
    )
    

    return encoded_pair_1, encoded_pair_2
model = ShroomformerV2()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs=100
best_val_loss = float('inf')
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters in the model:", total_params)
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences, labels = data
        optimizer.zero_grad()
        sentence1,sentence2=encode_pair(sentences)
        outputs=model(sentence1,sentence2)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_data in val_dataloader:
            val_sentences, val_labels = val_data
            val_sentence1, val_sentence2 = encode_pair(val_sentences)
            val_outputs = model(val_sentence1, val_sentence2)
            val_loss += criterion(val_outputs, val_labels.float()).item()
    val_loss /= len(val_dataloader)

    print(f'Epoch {epoch+1} | Train Loss: {running_loss / len(train_dataloader)} | Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        torch.save(model.state_dict(), "best_model_v2_lr1.pth")
        best_val_loss = val_loss


Total parameters in the model: 218964480


  0%|          | 0/100 [00:00<?, ?it/s]

Batch: 1
Batch: 101
Batch: 201
Batch: 301
Epoch 1 | Train Loss: 0.04102387403448423 | Validation Loss: 0.03369533416240456


  1%|          | 1/100 [08:55<14:43:51, 535.67s/it]

Batch: 1
Batch: 101
Batch: 201
Batch: 301
Epoch 2 | Train Loss: 0.020565950364754018 | Validation Loss: 0.029373987875086195


...

Epoch 10 | Train Loss: 0.0033531822940050106 | Validation Loss: 0.029611960846058865
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 11%|█         | 11/100 [1:35:39<13:06:45, 530.40s/it]

Epoch 11 | Train Loss: 0.0032314999809993123 | Validation Loss: 0.030233834315329156
Batch: 1


 11%|█         | 11/100 [1:36:46<13:02:59, 527.86s/it]


KeyboardInterrupt: 

In [92]:
torch.save(model.state_dict(), "model_v2_lr1.pth")
lrs={"lr1":0.00001}

In [93]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
    device = torch.device("cpu")
    

else:
    device = torch.device("mps")

In [101]:

from transformers import DistilBertTokenizer, DistilBertModel
class ShroomformerV2(nn.Module):
    def __init__(self):
        super(ShroomformerV2, self).__init__()
        self.twin1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.twin2 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    
    def mean_pooling(self,word_embeddings, attention_mask):
        token_embeddings = word_embeddings[0] 
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

  
    def forward(self, sentence1_encoded, sentence2_encoded):
        word_embeddings_1 = self.twin1(**sentence1_encoded)
        sentence_embeddings_1=self.mean_pooling(word_embeddings_1, sentence1_encoded["attention_mask"])
        word_embeddings_2 = self.twin2(**sentence2_encoded)
        sentence_embeddings_2=self.mean_pooling(word_embeddings_2, sentence2_encoded["attention_mask"])
        similarity_score = self.cos(sentence_embeddings_1,sentence_embeddings_2)
        return similarity_score
       


def encode_pair(pair, padding=True, truncation=True, max_length=128):
    tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    encoded_pair_1 = tokenizer(
        pair[0], 
        padding=padding, 
        truncation=truncation, 
        max_length=max_length, 
        return_tensors="pt"
    )
    encoded_pair_2 = tokenizer(
        pair[1],
        padding=padding,
        truncation=truncation,
        max_length=max_length,
        return_tensors="pt"
    )
    return encoded_pair_1, encoded_pair_2
model = ShroomformerV2()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs=100
best_val_loss = float('inf')
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters in the model:", total_params)
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences, labels = data
        optimizer.zero_grad()
        sentence1,sentence2=encode_pair(sentences)
        labels = labels.float()
        outputs=model(sentence1,sentence2)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for val_data in val_dataloader:
            val_sentences, val_labels = val_data
            val_sentence1, val_sentence2 = encode_pair(val_sentences)
            val_labels = val_labels.float()
            val_outputs = model(val_sentence1, val_sentence2)
            val_loss += criterion(val_outputs, val_labels).item()
    val_loss /= len(val_dataloader)

    print(f'Epoch {epoch+1} | Train Loss: {running_loss / len(train_dataloader)} | Validation Loss: {val_loss}')

    if val_loss < best_val_loss:
        torch.save(model.state_dict(), "best_model_v2_lr1.pth")
        best_val_loss = val_loss


Total parameters in the model: 132725760


  0%|          | 0/100 [00:00<?, ?it/s]

Batch: 1
Batch: 101
Batch: 201
Batch: 301
Epoch 1 | Train Loss: 0.03836502712996056 | Validation Loss: 0.03810320022773552


  1%|          | 1/100 [04:25<7:18:41, 265.87s/it]

Batch: 1
Batch: 101
Batch: 201
Batch: 301
Epoch 2 | Train Loss: 0.017532667429703806 | Validation Loss: 0.03760222710193472


...

Epoch 70 | Train Loss: 0.0010172858493913534 | Validation Loss: 0.06916271617754977
Batch: 1
Batch: 101
Batch: 201
Batch: 301


 71%|███████   | 71/100 [5:53:02<3:52:44, 481.54s/it]

Epoch 71 | Train Loss: 0.000997507691580621 | Validation Loss: 0.0707069143177347
Batch: 1


 71%|███████   | 71/100 [5:55:06<2:25:02, 300.09s/it]


KeyboardInterrupt: 

In [103]:
torch.save(model.state_dict(),"model_lr_e5.pth")

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample
from torch.utils.data import DataLoader

train_examples = []
for i,row in df_cosine_labels.iterrows():
    train_examples.append(InputExample(texts=[row["sentence1"], row["sentence2"]],label=row["label"]))
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

val_examples = []
for i,row in validation_cosine_labels.iterrows():
    val_examples.append(InputExample(texts=[row["sentence1"], row["sentence2"]],label=row["label"]))


val_dataloader = DataLoader(val_examples, shuffle=True, batch_size=16)

In [110]:

from transformers import DistilBertTokenizer, DistilBertModel
from sentence_transformers import SentenceTransformer, models
from torch import nn
class ShroomformerV2(nn.Module):
    def __init__(self):
        super(ShroomformerV2, self).__init__()
        word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
        pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
        downsampler_model = models.Dense(
            in_features=pooling_model.get_sentence_embedding_dimension(),
            out_features=256,
            activation_function=nn.Tanh())
        self.twin1 = SentenceTransformer(modules=[word_embedding_model, pooling_model, downsampler_model])
        self.twin2 = SentenceTransformer(modules=[word_embedding_model, pooling_model, downsampler_model])
        self.cos = nn.CosineSimilarity(dim=1, eps=1e-6)
    

    def forward(self, sentence1_encoded, sentence2_encoded):
        sentence1_embeddings =self.twin1()
        sentence2_embeddings = self.twin2(**sentence2_encoded)
        
        similarity_score = self.cos(sentence1_embeddings,sentence2_embeddings)
        return similarity_score
       



model = ShroomformerV2()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
num_epochs=1
best_val_loss = float('inf')
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total parameters in the model:", total_params)
for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0
    model.train()
    for i, data in enumerate(train_dataloader, 0):
        if i%100==0:
            print(f"Batch: {i+1}")
        sentences,labels=data
   

     
        optimizer.zero_grad()
    #     labels = labels.float()
        outputs=model(sentence1,sentence2)
        print(outputs)
        break
    #     loss = criterion(outputs, labels)
    #     loss.backward()
    #     optimizer.step()
    #     running_loss += loss.item()

    # model.eval()
    # val_loss = 0.0
    # with torch.no_grad():
    #     for val_data in val_dataloader:
    #         val_sentences, val_labels = val_data
    #         val_sentence1, val_sentence2 = encode_pair(val_sentences)
    #         val_labels = val_labels.float()
    #         val_outputs = model(val_sentence1, val_sentence2)
    #         val_loss += criterion(val_outputs, val_labels).item()
    # val_loss /= len(val_dataloader)

    # print(f'Epoch {epoch+1} | Train Loss: {running_loss / len(train_dataloader)} | Validation Loss: {val_loss}')

    # if val_loss < best_val_loss:
    #     torch.save(model.state_dict(), "best_model_v2_lr1.pth")
    #     best_val_loss = val_loss


Total parameters in the model: 109679104


  0%|          | 0/1 [00:00<?, ?it/s]

Batch: 1
{'input_ids': tensor([[  101,  1037,  2450,  4472,  1037,  3538,  1997,  6240,  2007, 13724,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101, 12849,  3549,  2180,  1005,  1056,  3013,  3740,  6687,  9021,
          4804,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  2048,  4584,  3322,  2056,  2055,  3998,  3667,  2018,  2042,
          4727,  2012,  6079,  5324,  1999,  2538,  2163,  1012,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  1996,  2500,  2020,  2445,  4809,  1997,  1000,  2852,  1012,
         21087,  1005,  2047,  

TypeError: Sequential.forward() got an unexpected keyword argument 'input_ids'

In [112]:
from transformers import AutoTokenizer, AutoModel
import torch


# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


# Sentences we want sentence embeddings for
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog.",
]


word_embedding_model = models.Transformer("bert-base-uncased", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
downsampler_model = models.Dense(
            in_features=pooling_model.get_sentence_embedding_dimension(),
            out_features=256,
            activation_function=nn.Tanh())
twin1 = SentenceTransformer(modules=[word_embedding_model, pooling_model, downsampler_model])
# Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Tokenize sentences
encoded_input = tokenizer(
    sentences, padding=True, truncation=True, max_length=128, return_tensors="pt"
)
print(encoded_input)
# Compute token embeddings
with torch.no_grad():
    model_output = twin1(**encoded_input)

# Perform pooling. In this case, mean pooling
sentence_embeddings = mean_pooling(model_output, encoded_input["attention_mask"])

{'input_ids': tensor([[  101,  2023,  7705, 19421,  7861,  8270,  4667,  2015,  2005,  2169,
          7953,  6251,   102],
        [  101, 11746,  2024,  2979,  2004,  1037,  2862,  1997,  5164,  1012,
           102,     0,     0],
        [  101,  1996,  4248,  2829,  4419, 14523,  2058,  1996, 13971,  3899,
          1012,   102,     0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])}


TypeError: Sequential.forward() got an unexpected keyword argument 'input_ids'